In [1]:
from train.train_utils import train
from configs.configs_utils import update_config
from data.data_utils import (get_dataloader, get_distillation_dataloader)
from train.tools import (setup, setup_environ_flags, clear_gpu_cache)
from models.models_utils import  get_optimizer
from models.xrag_models_utils import get_xrag_models
from data.data_utils import get_dataloader
import torch

/home/brimmann/works/llm-recipes-2/.venv/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/brimmann/works/llm-recipes-2/.venv/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have ha

In [6]:
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig
from configs.fsdp import fsdp_config as FsdpConfig

dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
fsdp_config = DistillationConfig()
train_config.model_name = "google/gemma-3-1b-it"
distill_config.model_name = "Hannibal046/xrag-7b"
train_config.batch_size_training = 1
train_config.distillation = True
train_config.num_workers_dataloader = 1
dataset_config.file = "data/loaders/squad-v2-sampled.py"

In [3]:
student_tokenizer, teacher_tokenizer, model = get_xrag_models()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of XGemmaForCausalLM were not initialized from the model checkpoint at google/gemma-3-1b-it and are newly initialized: ['projector.projector.0.bias', 'projector.projector.0.weight', 'projector.projector.2.bias', 'projector.projector.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in student model:
projector.projector.0.weight
projector.projector.0.bias
projector.projector.2.weight
projector.projector.2.bias


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


initializing xRAG distillation...


In [7]:
teacher_train_dataloader, teacher_eval_dataloader = get_dataloader(dataset_config, train_config, teacher_tokenizer, 0, distill_config)
student_train_dataloader, student_eval_dataloader = get_dataloader(dataset_config, train_config, student_tokenizer, 0, distill_config)

--> Training Set Length = 1000
--> Validation Set Length = 250
--> Training Set Length = 1000
--> Validation Set Length = 250


In [8]:
for step, batch in enumerate(teacher_train_dataloader):
    print(batch)
    if step == 2:
        break

{'input_ids': tensor([[  733, 16289, 28793, 24316, 28747, 32001, 28725,   690,   835,  2825,
         28747, 28792, 28748, 16289, 28793,  8712,  2372,  9782]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'retrieval_embeds': tensor([[ 4.0625,  0.7031,  1.4453,  ...,  5.5312, -2.8750, -0.6367]]), 'labels': tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, 8712, 2372, 9782]])}
{'input_ids': tensor([[  733, 16289, 28793, 24316, 28747, 32001, 28725,   690,   835,  2825,
         28747, 28792, 28748, 16289, 28793, 28705, 28740, 28723,  6711,   374,
          6899,  2970,  3368,    13,    13, 28750, 28723, 10668,  2970,  3368,
            13,    13, 28770, 28723,  3187,   645,  1060,    13,    13, 28781,
         28723,  3187,   645, 10228,    13,    13, 28782, 28723,  3187,   645,
          6370,    13,    13, 28784, 28723,  3187,   645, 14636,    13,    13,
         28787, 28723,  3187,   645,